# <font color=red>`Assignment # 4`</font>
##    <font color=red>Submitted By:</font> M.Janisar
##    <font color=red>Registration No:</font> SP22-BCS-047
##    <font color=white>______________________________________</font>

## `Preprocessing & Feature Finding`     <font color=red></font>

In [1]:
import nltk
import spacy
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
from autocorrect import Speller
nlp = spacy.load("en_core_web_sm")
# nltk.download('averaged_perceptron_tagger')
    
    
        
short_pos = open("D:\\NLP\\pos.txt","r").read()
short_neg = open("D:\\NLP\\neg.txt","r").read()
short_nuet = open("D:\\NLP\\nue.txt","r").read()
all_words = []
documents = []

def preprocessing(text):
    tokens = [token.text.lower() for token in nlp(text) if not token.is_stop and token.is_alpha]
    lemmas = [token.lemma_ for token in nlp(' '.join(tokens))]
    return lemmas


for p in short_pos.split('\n'):
    documents.append((p, "pos"))
    lemmas = preprocessing(p)
    for w in lemmas:
        all_words.append(w)

for p in short_neg.split('\n'):
    documents.append((p, "neg"))
    lemmas = preprocessing(p)
    for w in lemmas:
         all_words.append(w)

for p in short_nuet.split('\n'):
    documents.append((p, "nue"))
    lemmas = preprocessing(p)
    for w in lemmas:
         all_words.append(w)


save_documents = open("documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()


all_words = nltk.FreqDist(all_words)


word_features = list(all_words.keys())[:5000]


save_word_features = open("word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()


def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]

featuresets_save = open("featuressss.pickle","wb")
pickle.dump(featuresets, featuresets_save)
featuresets_save.close()

random.shuffle(featuresets)
print(len(featuresets))

testing_set = featuresets[1000:]
training_set = featuresets[:1000]



20003


## `Preprocessing & Feature Finding`     <font color=red>[accuracy, precision, recall, f1]</font>

In [2]:
import nltk
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support

def evaluate_classifier(clf, test_set):
    predictions = [clf.classify(sample[0]) for sample in test_set]
    gold = [sample[1] for sample in test_set]
    accuracy = nltk.classify.accuracy(clf, test_set)
    precision, recall, f1, _ = precision_recall_fscore_support(gold, predictions, average='weighted')
    return accuracy, precision, recall, f1

results = {}

classifier = nltk.NaiveBayesClassifier.train(training_set)
accuracy, precision, recall, f1 = evaluate_classifier(classifier, testing_set)
results['Naive Bayes'] = [accuracy, precision, recall, f1]

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
accuracy, precision, recall, f1 = evaluate_classifier(MNB_classifier, testing_set)
results['Multinomial NB'] = [accuracy, precision, recall, f1]

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
accuracy, precision, recall, f1 = evaluate_classifier(BernoulliNB_classifier, testing_set)
results['Bernoulli NB'] = [accuracy, precision, recall, f1]

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
accuracy, precision, recall, f1 = evaluate_classifier(LogisticRegression_classifier, testing_set)
results['Logistic Regression'] = [accuracy, precision, recall, f1]

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
accuracy, precision, recall, f1 = evaluate_classifier(LinearSVC_classifier, testing_set)
results['Linear SVC'] = [accuracy, precision, recall, f1]

SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
accuracy, precision, recall, f1 = evaluate_classifier(SGDC_classifier, testing_set)
results['SGD'] = [accuracy, precision, recall, f1]

average_scores = {clf: sum(metrics)/len(metrics) for clf, metrics in results.items()}
best_classifier = max(average_scores, key=average_scores.get)

print("{:<20} {:<10} {:<10} {:<10} {:<10} {:<10}".format('Classifier', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'Average'))
for clf, metrics in results.items():
    accuracy, precision, recall, f1 = metrics
    average = average_scores[clf]
    print("{:<20} {:.2f}      {:.2f}      {:.2f}      {:.2f}      {:.2f}".format(clf, accuracy * 100, precision * 100, recall * 100, f1 * 100, average * 100))

print("\nBest classifier based on average score: {}".format(best_classifier))


c:\Users\Ibrahim Qureshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ibrahim Qureshi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Classifier           Accuracy   Precision  Recall     F1 Score   Average   
Naive Bayes          73.30      67.19      73.30      69.74      70.89
Multinomial NB       80.00      74.04      80.00      71.14      76.29
Bernoulli NB         80.01      64.02      80.01      71.13      73.79
Logistic Regression  78.40      65.82      78.40      70.77      73.35
Linear SVC           70.50      65.94      70.50      68.04      68.75
SGD                  68.81      66.14      68.81      67.38      67.78

Best classifier based on average score: Multinomial NB
